In [1]:
from itertools import combinations
from pathlib import Path
import warnings
import os
import torch.nn as nn
import torch
import copy
import math
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from numba import njit, prange
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib
import optiver2023
warnings.filterwarnings("ignore")
model_path ='/kaggle/input/optiver-models'

In [2]:
def generate_features(df):
    prices = ['reference_price', 'far_price', 'mid_price',
              'near_price', 'ask_price', 'bid_price', 'wap']
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    df['mid_price'] = (df['ask_price']+df['bid_price'])/2
    
    for i, a in enumerate(prices):
        for j, b in enumerate(prices):
            if i > j:
                df[f'{a}_{b}_imb'] = (df[a]-df[b])/(df[a]+df[b])

    for c in [prices, sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    df['spread'] = df['ask_price']-df['bid_price']
    df['price_pressure'] = df['imbalance_size']*df['spread']
    df['ask_amount'] = df['ask_price']*df['ask_size']
    df['bid_amount'] = df['bid_price']*df['bid_size']
    df['marketdepth'] = df['ask_size']+df['bid_size']
    df['liquidity_imbalance'] = (
        df['bid_size']-df['ask_size'])/df['marketdepth']
    df['matched_imbalance'] = (df['imbalance_size']-df['matched_size']) / \
        (df['imbalance_size']+df['matched_size'])
    df['wap_2'] = (df['ask_amount']+df['bid_amount'])/df['marketdepth']
    df['bboimb'] = (df['bid_amount']-df['ask_amount']) / \
        (df['ask_amount']+df['bid_amount'])
    df['price_imbalance'] = df['bid_price']/df['ask_price']
    df['size_imbalance'] = df['bid_size']/df['ask_size']
    df['market_urgency'] = df['spread'] * df['liquidity_imbalance']
    df['effectiveSpread'] = np.abs(df['wap']-df['mid_price'])/df['mid_price']
    df['shallowLIX'] = np.log(
        df['matched_size']/(df['ask_price']-df['bid_price']))
    df['imbalance_ratio'] = df['imbalance_size'] / \
        (df['imbalance_size']+df['matched_size'])
    df['logquoteslope'] = (np.log(df['ask_price'])-np.log(df['bid_price'])) / \
        (np.log(df['ask_size'])+np.log(df['bid_size']))
    df['wapm'] = (df['wap_2']-df['mid_price'])/df['mid_price']
    df['MCI'] = df['wapm']/(df['ask_amount']+df['bid_amount'])
    df['imb_size_with_flag'] = (2 * df['imbalance_size'] *
                                df['imbalance_buy_sell_flag'])/df['marketdepth']
    df['farrefliquidity'] = (
        df['far_price']/df['reference_price']-1)/df['matched_size']
    df['farmidliquidity'] = (
        df['far_price']/df['mid_price']-1)/df['matched_size']
    df['farnearliquidity'] = (
        df['far_price']/df['near_price']-1)/df['matched_size']
    df['farwapliquidity'] = (df['far_price']/df['wap']-1)/df['matched_size']
    df['depth_pressure'] = (df['ask_size']-df['bid_size']) * \
        (df['far_price']-df['near_price'])
    return df


def market_features(df_1day):
    weights = [
        0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
        0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
        0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
        0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04, 0.002, 0.001,
        0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
        0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
        0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
        0.02, 0.004, 0.006, 0.002, 0.02, 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
        0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
        0.004, 0.006, 0.006, 0.001, 0.04, 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
        0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
        0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
        0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
        0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
        0.04, 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02, 0.004, 0.002, 0.006, 0.02,
        0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
        0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
    ]
    weights = pd.Series(weights)
    weights.index.name = 'stock_id'

    market_feats = pd.DataFrame(
        index=df_1day.index.get_level_values(0).unique())
    wap = df_1day['wap'].unstack()
    market_feats['index_wap_std'] = wap.std(axis=1)
    stock_ret1 = 10000*(wap/wap.shift(1)-1)
    market_feats['index_ret1'] = stock_ret1.mean(axis=1)
    market_feats['index_ret1_std'] = stock_ret1.std(axis=1)
    market_feats['index_ret1_w'] = stock_ret1.mul(weights, axis=1).sum(axis=1)
    stock_ret6 = 10000*(wap/wap.shift(6)-1)
    market_feats['index_ret6'] = stock_ret6.mean(axis=1)
    market_feats['index_ret6_std'] = stock_ret6.std(axis=1)
    market_feats['index_ret6_w'] = stock_ret6.mul(weights, axis=1).sum(axis=1)
    ask_size = df_1day['ask_size'].unstack()
    bid_size = df_1day['bid_size'].unstack()
    market_feats['market_volume'] = ask_size.mean(axis=1)+bid_size.mean(axis=1)
    ask_price = df_1day['ask_price'].unstack()
    bid_price = df_1day['bid_price'].unstack()
    market_feats['market_spread'] = ask_price.mean(
        axis=1)-bid_price.mean(axis=1)
    market_feats['market_max_spread'] = ask_price.max(
        axis=1)-bid_price.min(axis=1)
    market_feats['mfd'] = market_feats.index // 60
    return market_feats


def cal_extra_fea(df_original_fea,
                  window=[6, 12],
                  diff_window=[1, 2, 3, 5, 10],
                  fac_to_remain=None,
                  SUB_MODE=False):
    '''
    df_original_fea: pd.Series, index为datetime和security, name为因子名称
    window: list
    fac_to_remain: list, 指保留的因子名称
     '''
    f = df_original_fea.name
    fea_wide = df_original_fea.unstack()
    # 用于生成本地因子
    if not SUB_MODE:
        feature = pd.DataFrame(index=df_original_fea.index)
        for w in diff_window:
            # diff
            fname = f'{f}_diff{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide-fea_wide.shift(w)
                feature[fname] = fea_tmp.stack(dropna=False)
            # mom
            fname = f'{f}_mom{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide/fea_wide.shift(w)-1
                feature[fname] = fea_tmp.stack(dropna=False)
        for w in window:
            # ma
            fname = f'{f}_ma{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).mean()
                feature[fname] = fea_tmp.stack(dropna=False)
            # qtlu
            fname = f'{f}_qtlu{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).quantile(0.8)
                feature[fname] = fea_tmp.stack(dropna=False)
            # qtld
            fname = f'{f}_qtld{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).quantile(0.2)
                feature[fname] = fea_tmp.stack(dropna=False)
            # rank
            fname = f'{f}_rank{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).rank(pct=True)
                feature[fname] = fea_tmp.stack(dropna=False)
            # std
            fname = f'{f}_std{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).std()
                feature[fname] = fea_tmp.stack(dropna=False)
            # max
            fname = f'{f}_max{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).max()
                feature[fname] = fea_tmp.stack(dropna=False)
            # low
            fname = f'{f}_low{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(w, min_periods=1).min()
                feature[fname] = fea_tmp.stack(dropna=False)
            # IMAX
            fname = f'{f}_imax{w}'

            def findMaxIdx(
                series): return series.shape[0] - series.reset_index(drop=True).idxmax()
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(
                    w, min_periods=1).apply(findMaxIdx) / w
                feature[fname] = fea_tmp.stack(dropna=False)
            # IMIN
            fname = f'{f}_imin{w}'

            def findMinIdx(
                series): return series.shape[0] - series.reset_index(drop=True).idxmin()
            if fac_to_remain is None or fname in fac_to_remain:
                fea_tmp = fea_wide.rolling(
                    w, min_periods=1).apply(findMinIdx) / w
                feature[fname] = fea_tmp.stack(dropna=False)
    # 用于推理
    else:
        feature = pd.DataFrame(index=fea_wide.columns.to_list())
        feature.index.name = 'stock_id'
        for w in diff_window:
            # diff
            fname = f'{f}_diff{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-1] - fea_wide.iloc[-(w+1)]
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # mom
            fname = f'{f}_mom{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-1] / fea_wide.iloc[-(w+1)] - 1
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
        for w in window:
            # ma
            fname = f'{f}_ma{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    # fea_tmp = fea_wide.iloc[-w:].mean()
                    fea_tmp = fea_wide.iloc[-w:].rolling(
                        w, min_periods=1).mean().iloc[-1, :]
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # qtlu
            fname = f'{f}_qtlu{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].quantile(0.8)
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # qtld
            fname = f'{f}_qtld{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].quantile(0.2)
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # rank
            fname = f'{f}_rank{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].rank(pct=True).iloc[-1]
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # std
            fname = f'{f}_std{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    # fea_tmp = fea_wide.iloc[-w:].std()
                    fea_tmp = fea_wide.iloc[-w:].rolling(
                        w, min_periods=1).std().iloc[-1, :]
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # max
            fname = f'{f}_max{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].max()
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # low
            fname = f'{f}_low{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].min()
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # IMAX
            fname = f'{f}_imax{w}'

            def findMaxIdx(
                series): return series.shape[0] - series.reset_index(drop=True).idxmax()
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].apply(findMaxIdx) / w
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
            # IMIN
            fname = f'{f}_imin{w}'

            def findMinIdx(
                series): return series.shape[0] - series.reset_index(drop=True).idxmin()
            if fac_to_remain is None or fname in fac_to_remain:
                try:
                    fea_tmp = fea_wide.iloc[-w:].apply(findMinIdx) / w
                    feature[fname] = fea_tmp
                except:
                    feature[fname] = np.nan
    return feature


@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

def huber_approx_obj(dtrain, preds):
    d = preds - dtrain  
    h = 1 #h is delta in the formula  
    scale = 1 + (d / h) ** 2  
    scale_sqrt = np.sqrt(scale)  
    grad = d / scale_sqrt  
    hess = 1 / scale / scale_sqrt  
    return grad, hess

In [4]:
from copy import deepcopy

feature_lst0 = pd.read_csv(Path(model_path, 'feature_importance0_final.csv'), 
                           index_col=0).index.to_list()
feature_lst0_RSR = deepcopy(feature_lst0)
if 'mfd' in feature_lst0_RSR:
    feature_lst0_RSR.remove('mfd')
    feature_lst0_RSR = ['mfd',] + feature_lst0_RSR    # 把需要embedding的放在最前面
print(len(feature_lst0), feature_lst0[:3], feature_lst0[-3:])


feature_lst1 = pd.read_csv(Path(model_path, 'feature_importance1_final.csv'), 
                           index_col=0).index.to_list()
###################################
original_RSR_feature_lst1 = deepcopy(feature_lst1)
if 'mfd' in original_RSR_feature_lst1:
    original_RSR_feature_lst1.remove('mfd')
    original_RSR_feature_lst1 = ['mfd',] + original_RSR_feature_lst1    # 把需要embedding的放在最前面
###################################
feature_lst1 = [x for x in feature_lst1 if x not in feature_lst0]
feature_lst1_RSR = deepcopy(feature_lst1)
if 'mfd' in feature_lst1_RSR:
    feature_lst1_RSR.remove('mfd')
    feature_lst1_RSR = ['mfd',] + feature_lst1_RSR    # 把需要embedding的放在最前面
print(len(feature_lst1), feature_lst1[:3], feature_lst1[-3:])


feature_lst01 = list(set(feature_lst0+feature_lst1))
print(len(feature_lst01), feature_lst01[:3], feature_lst01[-3:])


norm_col_lst0 = ['index_wap_std', 'index_ret1', 'index_ret1_std', 'index_ret1_w', 'index_ret6', 
                 'index_ret6_std', 'index_ret6_w', 'market_volume', 'market_spread', 'market_max_spread']
norm_col_lst1 = [x for x in norm_col_lst0 if x not in feature_lst0]

203 ['effectiveSpread', 'MCI', 'wap_reference_price_imb'] ['reference_price_std12', 'ask_price_diff2', 'bid_price_reference_price_imb_diff2']
89 ['wap_near_price_imb', 'bid_price_near_price_imb', 'wap_near_price_imb_diff1'] ['wap_ask_price_imb_max12', 'near_price_reference_price_imb_diff10', 'farmidliquidity_diff2']
292 ['matched_size_bid_size_imbalance_size_imb2_diff10', 'reference_price_ask_price_bid_price_imb2_std12', 'mfd'] ['wap_ask_price_imb_qtlu12', 'spread', 'wap_reference_price_imb_low6']


In [6]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [7]:
counter = 0
all_feats = {}
all_results = []
for (test, revealed_targets, sample_prediction) in iter_test:
    date = test['date_id'].unique()[0]
    if all_feats.get(date,0) == 0:
        all_feats[date] = []
    second = test['seconds_in_bucket'].unique()[0]
    currently_scored = test['currently_scored'].unique()[0]
    if not currently_scored:  # 不进行调试
        sample_prediction['target'] = 0.0
        env.predict(sample_prediction)
        counter += 1
        continue
    print(date, second, currently_scored)
    n_test = test.set_index(['seconds_in_bucket', 'stock_id'])
    del n_test['date_id'], n_test['row_id'], n_test['currently_scored']
    # raw features
    if second == 0:
        features = generate_features(n_test)
        del features['imbalance_buy_sell_flag']
        feat = features.copy()
    else:
        feat = generate_features(n_test)
        del feat['imbalance_buy_sell_flag']
        features = pd.concat([features, feat], axis=0)
    # expand features
    tmp_features = []
    for f in features.columns:
        fac_to_remain=feature_lst0 if second<300 else feature_lst01
        tmp = cal_extra_fea(df_original_fea=features[f], 
                            fac_to_remain=fac_to_remain, 
                            SUB_MODE=True)
        tmp_features.append(tmp)
    tmp_features = pd.concat(tmp_features, axis=1)
    tmp_features = tmp_features.reset_index()
    tmp_features['seconds_in_bucket'] = second
    tmp_features = tmp_features.set_index(['seconds_in_bucket', 'stock_id'])
    # market factors
    market_feats = market_features(feat)
    # merge factors
    feat = feat.merge(right=market_feats, 
                      left_on='seconds_in_bucket', 
                      right_index=True, 
                      how='left')
    feat = pd.concat([feat, tmp_features], axis=1)
    feat = feat.replace([np.inf, -np.inf], np.nan)
    all_feats[date].append(feat)
    # predict
    if second < 80:
        tree_predictions_0 = ensemble_models.ensemble_tree_predict(feat, 0)
        sample_prediction['target'] = tree_predictions_0
    elif second < 300:
        tree_predictions_0 = ensemble_models.ensemble_tree_predict(feat, 0)
        nn_predictions_0 = ensemble_models.ensemble_nn_predict(all_feats, second, 0)
#         print(nn_predictions_0.shape)
        sample_prediction['target'] = (tree_predictions_0+nn_predictions_0)/2
#         sample_prediction['target'] = nn_predictions_0
    elif second < 380:
        tree_predictions_0 = ensemble_models.ensemble_tree_predict(feat, 0)
        tree_predictions_1 = ensemble_models.ensemble_tree_predict(feat, 1)
        sample_prediction['target'] = tree_predictions_0+tree_predictions_1
    else:
        tree_predictions_0 = ensemble_models.ensemble_tree_predict(feat, 0)
        tree_predictions_1 = ensemble_models.ensemble_tree_predict(feat, 1)
        nn_predictions_0 = ensemble_models.ensemble_nn_predict(all_feats, second, 0)
#         nn_predictions_1 = ensemble_models.ensemble_nn_predict(all_feats, second, 1)
        predictions_0 = (tree_predictions_0+nn_predictions_0)/2
#         predictions_1 = (tree_predictions_1+nn_predictions_1)/2
#         predictions_0 = nn_predictions_0
        predictions_1 = tree_predictions_1
        sample_prediction['target'] = predictions_0+predictions_1
    env.predict(sample_prediction)
    counter += 1
print(counter)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
165
